In [1]:
# import dependencies
import pandas as pd 
import requests
import time
import pymongo
from  tqdm import tqdm
from pprint import pprint

#import local libraries
import functions_mongo as mdb
import funtions_webscraping as web

In [2]:
##### Initialize Global Variables #####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

#create collection to hold SEP TOC
toc_collection = db.sep_win2019_toc



In [ ]:
# scrape TOC from SEP 

print("Starting: SEP TOC Scrape\n")

#TOC to scrape
search_url = "https://plato.stanford.edu/archives/win2019/contents.html"

#url to generate absolute paths from
base_url = "https://plato.stanford.edu/archives/win2019/"

#current TOC edition
toc_edition = "Winter 2019"

#hold all individual TOC entries 
toc_list = []

#scrape selected webpage and save HTML file
soup = web.scrape_web_page(search_url, "data/win2019_toc.html")

#get all links with an HREF attribute in the content page
links_in_toc = soup.find(id="content").findAll("a", href=True)

file_toclist = open("html_files/sep_toc_list.txt","w",encoding="UTF-8")
file_links = open("html_files/sep_toc_links.txt", "w", encoding="UTF-8")

for link in links_in_toc:
    link_text = link.get_text()
    link_href = link["href"]

    if link_href != '#pagetopright':
        #create dictionary to append to mongo
        entry = {"link_text": link_text, "base_url":base_url, "link_url":link_href}
        if entry not in toc_list:
            toc_list.append(entry)
            file_toclist.write(f"{link_text}:{link_href}\n")
            file_links.write(f"{link}\n")

#insert into Mongo
toc_collection.insert_many(toc_list)

print("Completed: SEP TOC Scrape\n")


In [3]:
articles = toc_collection.find({'link_text': {'$regex': '^Chinese'}}).sort('link_text')
for article in articles:
    print(article)


{'_id': ObjectId('5e310663c39f6ace9361b848'), 'link_text': 'Chinese Philosophy: Chan Buddhism', 'base_url': 'https://plato.stanford.edu/archives/win2019/', 'link_url': 'entries/buddhism-chan/'}
{'_id': ObjectId('5e310663c39f6ace9361bc1e'), 'link_text': 'Chinese Philosophy: Chinese medicine', 'base_url': 'https://plato.stanford.edu/archives/win2019/', 'link_url': 'entries/chinese-phil-medicine/'}
{'_id': ObjectId('5e310663c39f6ace9361b84d'), 'link_text': 'Chinese Philosophy: Huayan Buddhism', 'base_url': 'https://plato.stanford.edu/archives/win2019/', 'link_url': 'entries/buddhism-huayan/'}
{'_id': ObjectId('5e310663c39f6ace9361bc60'), 'link_text': 'Chinese Philosophy: Mohism', 'base_url': 'https://plato.stanford.edu/archives/win2019/', 'link_url': 'entries/mohism/'}
{'_id': ObjectId('5e310663c39f6ace9361bc61'), 'link_text': 'Chinese Philosophy: Mohist Canons', 'base_url': 'https://plato.stanford.edu/archives/win2019/', 'link_url': 'entries/mohist-canons/'}
{'_id': ObjectId('5e310663c39

In [4]:
# articles = toc_collection.find().sort('link_text')
pages = toc_collection.find({'link_text': {'$regex': '^Chinese'}}).sort('link_text')

for page in tqdm(pages, desc="Processing"):
    base_url = page['base_url']
    link_url = page['link_url']
    slash_pos = link_url.rfind("/",0,len(link_url)-1)
    page_name = link_url[slash_pos+1:len(link_url)-1]
    page_url = f"{base_url}{link_url}"
    html_save_as = f"html_files/sep/{page_name}.html"
    soup = web.scrape_web_page(page_url, html_save_as)




Processing: 0it [00:00, ?it/s]html_files/sep/buddhism-chan.html
Processing: 1it [00:17, 17.19s/it]html_files/sep/chinese-phil-medicine.html
Processing: 2it [00:34, 17.13s/it]html_files/sep/buddhism-huayan.html
Processing: 3it [00:53, 17.74s/it]html_files/sep/mohism.html
Processing: 4it [01:21, 20.97s/it]html_files/sep/mohist-canons.html
Processing: 5it [01:40, 20.40s/it]html_files/sep/qing-philosophy.html
Processing: 6it [01:58, 19.53s/it]html_files/sep/buddhism-tiantai.html
Processing: 7it [02:16, 19.15s/it]html_files/sep/ethics-chinese.html
Processing: 8it [02:40, 20.55s/it]html_files/sep/chinese-change.html
Processing: 9it [03:00, 20.49s/it]html_files/sep/comparphil-chiwes.html
Processing: 10it [03:18, 19.69s/it]html_files/sep/chinese-phil-medicine.html
Processing: 11it [04:04, 27.61s/it]html_files/sep/chinese-room.html
Processing: 12it [04:23, 21.99s/it]
